# Working with Tensoflow

**Objective**: Build a deep learning model that can learn the alphabet.

**Agenda**
1. Visualizing the end model
2. Short theroretical recap of LSTM networks
3. Implementation
   - Encoding the data
   - Building the model
   - Training
4. Using `Tensorboard` to overview model graph and `loss`/`accuracy` evolution
   - Structuring the graph with name scopes
   - Adding model summaries
   - Bonus: debugging data with `text summary`

## Visualizing the end model

Before building a model it's good to have a mental picture of what we want to achieve. In our case we'll be building a model that will receive a sequence of letters and will output a vector of 26 components where each element is the probability of that letter being the next letter in the sequence.

![Model architecture](./img/model.png)

In [2]:
import numpy as np
import tensorflow as tf

[LSTM]:Hochreiter, S., & Schmidhuber, J. (1997). Long short-term memory. Neural computation, 9(8), 1735-1780.